# Multiprocessing

In [3]:
def square(x):
    return x**2

In [4]:
data=[i for i in range(1000)]

In [5]:
%time
seq=[square(x) for x in data]
seq[:10]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.29 µs


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [7]:
import multiprocessing as mp
import numpy as np

In [8]:
mp.cpu_count()

4

In [14]:
%time
pool=mp.Pool(mp.cpu_count())  # todos los cores
res=pool.map(square, data)
res[:10]

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 27.9 µs


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Process ForkPoolWorker-22:
Process ForkPoolWorker-21:
Process ForkPoolWorker-23:
Process ForkPoolWorker-24:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocess

In [15]:
array=np.random.randint(0, 10, size=[200, 5000])
data=array.tolist()

In [16]:
def rango(r, mini, maxi):
    c=0
    for e in r:
        if mini<=e<=maxi:
            c+=1
    return c

In [17]:
%time res=[rango(e, 4, 8) for e in data]
res[:10]

CPU times: user 84.1 ms, sys: 3.29 ms, total: 87.4 ms
Wall time: 94.7 ms


[2497, 2480, 2483, 2497, 2469, 2540, 2536, 2470, 2513, 2492]

In [18]:
pool=mp.Pool()

%time res=pool.starmap(rango, [(e, 4, 8) for e in data])
pool.close()

res[:10]

CPU times: user 25.3 ms, sys: 2.05 ms, total: 27.3 ms
Wall time: 56.2 ms


[2497, 2480, 2483, 2497, 2469, 2540, 2536, 2470, 2513, 2492]

In [19]:
# asincrono

def rango2(i, r, mini, maxi):
    c=0
    
    for e in r:
        if mini<=e<=maxi:
            c+=1
            
    return i, c

In [20]:
res=[]

def colecta(result):
    global res
    res.append(result)

In [32]:
pool=mp.Pool(3)  # tres nucleos

%time
for i,e in enumerate(data):
    pool.apply_async(rango2, args=(i,e,4,8), callback=colecta)
    
pool.close()
pool.join()


res.sort(key=lambda x: x[0])

res[:10]

CPU times: user 4 µs, sys: 5 µs, total: 9 µs
Wall time: 34.1 µs


[(0, 2497),
 (0, 2497),
 (1, 2480),
 (1, 2480),
 (2, 2483),
 (2, 2483),
 (3, 2497),
 (3, 2497),
 (4, 2469),
 (4, 2469)]

In [26]:
sum([x[1] for x in res])

499913

In [30]:
sum([e for e in data[0] if 4<=e<=8])

15089

In [31]:
array.shape

(200, 5000)

In [ ]:
res=[]
%time
for e in data:
    for f in e:
        if 4<=f<=8:
            res.append(f)

In [38]:
# con pandas

import pandas as pd

In [39]:
df=pd.DataFrame(np.random.randint(3, 10, size=[5, 2]))
df

,0,1
0,5,8
1,9,3
2,7,9
3,7,7
4,9,3


In [40]:
def hipo(c):
    return (c[1]**2+c[2]**2)**0.5

In [41]:
with mp.Pool(3) as pool:
    res=pool.imap(hipo, df.itertuples(name=None), chunksize=10)
    out=[round(e, 2) for e in res]
out

[9.43, 9.49, 11.4, 9.9, 9.49]

In [42]:
hipo([0, 5, 8])

9.433981132056603

# Pathos

In [43]:
from pathos.multiprocessing import ProcessingPool as Pool

In [45]:
df=pd.DataFrame(np.random.randint(3, 10, size=[500, 2]))
df.head()

,0,1
0,6,9
1,8,9
2,4,8
3,4,7
4,8,8


In [46]:
def fun(df):
    return df.shape

In [47]:
cores=mp.cpu_count()

df_split=np.array_split(df, cores, axis=0)

pool=Pool(cores)

%time

df_out=pd.DataFrame(np.vstack(pool.map(fun, df_split)))

pool.close()
pool.join()
pool.clear()

CPU times: user 4 µs, sys: 5 µs, total: 9 µs
Wall time: 18.1 µs


In [50]:
df_out

,0,1
0,125,2
1,125,2
2,125,2
3,125,2


In [51]:
df.head()

,0,1
0,6,9
1,8,9
2,4,8
3,4,7
4,8,8
